In [ ]:
import efficientnet.keras as efn
import tensorflow as tf
tf.config.run_functions_eagerly(True)

base_model = efn.EfficientNetB0(input_shape = (154, 154, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = tf.keras.layers.Dense(11, activation="sigmoid")(x)
model_final = tf.keras.Model(base_model.input, predictions)

model_final.compile(tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from PIL import Image
import os
import numpy as np


base_path = 'dataset/'
data = {}
for classes in os.listdir(base_path):
    img_path = base_path+classes+'/'
    cls = int(classes)
    if  data.get(cls) is None:
        data[cls] = []

    for images in os.listdir(img_path):
        fmt = images.split('.')

        if fmt[-1] == "jpg":
            id = fmt[0]
            image_path = img_path + images
            img = Image.open(image_path)
            width, height = img.size
            top_crop = 66
            bottom_crop = 20
            left_crop = 43
            right_crop = 43 
            box = (left_crop, top_crop, width-right_crop, height - bottom_crop)
            cropped_img = img.crop(box)
            data[cls].append(np.array(cropped_img))
            img.close()

n_train = 10
Y_train = np.empty((0), int)
Y_test = np.empty((0), int)
X_train = np.empty((0,154,154,3), float)
X_test = np.empty((0,154,154,3), float)

for label in data:
    if label != 90:
        n = len(data[label])
        Y_train = np.append(Y_train, [label]*n_train, axis=0)
        X_train = np.append(X_train, data[label][0:n_train], axis=0)
        n -= n_train
        Y_test = np.append(Y_test, [label]*n, axis=0)
        X_test = np.append(X_test, data[label][n_train:], axis=0)
    else:
        dtr = np.array(data[label][0])
        dtr = dtr[np.newaxis,:]
        dte = np.array(data[label][1])
        dte = dte[np.newaxis,:]
        Y_train = np.append(Y_train, [label], axis=0)
        X_train = np.append(X_train, dtr, axis=0)
        Y_test = np.append(Y_test, [label], axis=0)
        X_test = np.append(X_test, dte, axis=0)

Y1 = np.zeros((11, len(Y_train)))
for i in range(0,len(Y_train)):
    Y1[Y_train[i]//10,i] = 1

Y2 = np.zeros((11, len(Y_test)))
for i in range(0,len(Y_test)):
    Y2[Y_test[i]//10,i] = 1

eff_history = model_final.fit(x=X_train, y=Y1.T, epochs=200, batch_size=5)